# Installation:-

In [ ]:
# %%capture
!pip install chromadb epitran panphon numpy phonemizer 

## To install epitran dependencies:-

In [ ]:
!pip install chromadb epitran panphon numpy
# Install espeak-ng based on the operating system
import platform
from subprocess import check_output, CalledProcessError
if platform.system() == "Linux":
    !apt-get update
    !apt-get install -y espeak-ng
elif platform.system() == "Windows":
    try:
        check_output(['choco', 'install', 'espeak-ng'], text=True)
    except CalledProcessError as e:
        print(f"Error installing espeak-ng: {e}")

In [ ]:
!sudo apt-get update
!sudo apt-get install -y build-essential git wget
# Clone the Flite repository
!git clone https://github.com/festvox/flite.git
%cd flite
# Build Flite
!./configure && make
# Build lex_lookup
%cd testsuite
!make lex_lookup
# Move lex_lookup to a global path
!sudo cp lex_lookup /usr/local/bin

In [ ]:
%cd ..
%cd ..

/root/Vdb


/root/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
!pwd

/root/Vdb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# To IPA conversion:-

### 1) via epitran(flite backend):-

In [2]:
from epitran import Epitran

# Initialize Epitran for English
english_epi = Epitran('eng-Latn')

def english_to_ipa(text):
    """Converts English text to IPA using Epitran."""
    try:
        return english_epi.transliterate(text)
    except Exception as e:
        return f"Error: {e}"

# Initialize Epitran for Hindi
hindi_epi = Epitran('hin-Deva')

def hindi_to_ipa(text):
    """Converts Hindi text to IPA using Epitran."""
    try:
        return hindi_epi.transliterate(text)
    except Exception as e:
        return f"Error: {e}"

In [44]:
print(english_to_ipa("aarthy"))

ɑɹθi


### 2) via phonemizer(espeak backend):-

In [ ]:
from phonemizer import phonemize

def english_to_ipa(text):
    """Converts English text to IPA using phonemiser with espeak-ng."""
    try:
        ipa = phonemize(text, language='en-us', backend='espeak', strip=True)
        return ipa
    except Exception as e:
        return f"Error: {e}"

def hindi_to_ipa(text):
    """Converts Hindi text to IPA using epitran."""
    try:
        return hindi_epi.transliterate(text)
    except Exception as e:
        return f"Error: {e}"

In [2]:
print(english_to_ipa("aarthy"))

ɑɹθi


# Database actions:-

In [1]:
import panphon
import panphon.distance 
import numpy as np
from chromadb.config import Settings
from chromadb import Client
import chromadb
import os
# Initialize PanPhon
ft = panphon.FeatureTable()
from panphon import distance
dst = distance.Distance()

In [3]:
from alt_ipa import alt_ipa

In [2]:
# Specify the persistence directory
persist_directory = "Vdb(epi)"
os.makedirs(persist_directory, exist_ok=True)
client = chromadb.PersistentClient(path=persist_directory)

In [ ]:
import pandas as pd

# def create_dataset_from_excel(file_path, sheet_name=0):
#     """
#     Reads an Excel file and creates a dataset for ChromaDB.

#     Args:
#     - file_path (str): Path to the Excel file.
#     - sheet_name (str or int): Sheet name or index to read from the Excel file.

#     Returns:
#     - names (list): List of names from the Excel file.
#     - ipas (list): List of IPA representations of the names.
#     """
#     # Read the Excel file
#     df = pd.read_excel(file_path, sheet_name=sheet_name)

#     # Assuming the Excel file has a column named 'Name'
#     names = df['Name'].tolist()

#     # Convert names to IPA
#     ipas = [english_to_ipa(name) for name in names]

#     return names, ipas

# # Example usage
# file_path = 'path_to_your_excel_file.xlsx'
# names, ipas = create_dataset_from_excel(file_path)

# # Add pronunciations to the ChromaDB collection
# embedder(names, ipas)
# # Create a new ChromaDB collection
# new_collection = client.create_collection(name="new_excel_collection")

# Add pronunciations to the new ChromaDB collection
# embedder(names, ipas)
# Add all values of the Excel file to the collection
def add_excel_data_to_collection(file_path, sheet_name=0):
    """
    Reads an Excel file and adds data to the ChromaDB collection.

    Args:
    - file_path (str): Path to the Excel file.
    - sheet_name (str or int): Sheet name or index to read from the Excel file.
    """
    # Read the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Assuming the Excel file has a column named 'Name'
    names = df['Name'].tolist()

    # Convert names to IPA
    ipas = [english_to_ipa(name) for name in names]

    # Prepare metadata
    metadatas = df.to_dict(orient='records')

    # Prepare embeddings
    embeddings = [ipa2vec(ipa) for ipa in pronunciationipas]
    # embeddings = [ft.word_array(ipa) for ipa in ipas]
    # Add data to the collection
    collection.add(
        documents=names,
        embeddings=embeddings,
        metadatas=metadatas,
        ids=[str(i) for i in range(len(names))]
    )

# Example usage
add_excel_data_to_collection(file_path)

## To list collections:-

In [15]:
# Get a list of all collection names
collection_names = client.list_collections()

# Print the collection names
for name in collection_names:
    print(name)

Collection(name=200k_single_hindi_cosine_today)


## To create a new vdb:-

In [14]:
# Create a ChromaDB collection
# collection = client.create_collection( name="vdb_l2")
collection = client.create_collection( name="200k_single_hindi_cosine_today",metadata={"hnsw:space": "cosine"} )

## To delete a collection:-

In [14]:
client.delete_collection(name="vdb_fuzzified56k-200k_cosine")

## or load an existing db:-

In [6]:
#Final db:-
collection = client.get_collection("200k_single_hindi_cosine_today")

In [34]:
collection = client.get_collection("final_200k_fullname_cosine")

In [20]:
collection = client.get_collection("vdb_l2")

In [20]:
collection = client.get_collection("vdb_cosine")

In [4]:
collection = client.get_collection("vdb_277k_fullname_cosine")

In [15]:
# Get the total number of documents in the collection
total_documents = collection.count()
print(f"Total number of documents in the collection: {total_documents}")

Total number of documents in the collection: 0


## Function definitions:-

In [16]:
# Function to compute the average feature vector (same as before)
def ipa2vec(ipa):
    vectors = ft.word_to_vector_list(ipa, numeric=True)
    processed_vectors = np.array(vectors)
    if processed_vectors.ndim == 2:
        avg_vector = np.mean(processed_vectors, axis=0)
    else:
        avg_vector = processed_vectors
    return avg_vector.tolist()  # ChromaDB needs lists


In [32]:
# Function to compute the average feature vector (same as before)
def cipa2vec(ipa):
    vectors = ft.word_to_vector_list(ipa, numeric=True)
    processed_vectors = np.array(vectors)
    if processed_vectors.ndim == 2:
        avg_vector = np.concatenate(processed_vectors, axis=0)
    else:
        avg_vector = processed_vectors
    return avg_vector.tolist()  # ChromaDB needs lists


In [35]:
ipa=english_to_ipa("noel")
print(ipa)
print(ft.word_to_vector_list(ipa, numeric=True))
print(len(ft.word_to_vector_list(ipa, numeric=True)[0]))
print(cipa2vec(english_to_ipa("noel")))
print(len(cipa2vec(english_to_ipa("noel"))))

nowɛl
[[-1, 1, 1, -1, -1, -1, 1, -1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, 0], [1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 0, -1, 0, -1, -1, -1, 1, 1, -1, 1, -1, 0, 0], [-1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 0, 1, 1, -1, 1, 1, -1, 0, -1, 0, 0], [1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 0, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0], [-1, 1, 1, 1, -1, 1, -1, -1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, 0]]
24
[-1, 1, 1, -1, -1, -1, 1, -1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, 0, 1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 0, -1, 0, -1, -1, -1, 1, 1, -1, 1, -1, 0, 0, -1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 0, 1, 1, -1, 1, 1, -1, 0, -1, 0, 0, 1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 0, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, 1, 1, 1, -1, 1, -1, -1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, 0]
120


In [6]:

def embedder(names, pronunciations):
    """Adds pronunciations to the ChromaDB collection."""

    if len(names) != len(pronunciations):
        raise ValueError("Names and pronunciations lists must have the same length.")

    embeddings = [ipa2vec(ipa) for ipa in pronunciations]
    collection.add(
        documents=names,
        embeddings=embeddings,
        metadatas=[{"ipa": ipa} for ipa in pronunciations],  # Store IPA for reference
        ids=[str(i) for i in range(len(names))] # provide IDs to avoid issues
    )


In [41]:
# Get the total number of documents in the collection
total_documents = collection.count()
print(f"Total number of documents in the collection: {total_documents}")

Total number of documents in the collection: 404845


# Querrier:-

## without weighted levenshtien:-

In [18]:
def chromaquerrier(query_ipa, n_results=5):
    """Retrieves similar names from the ChromaDB vdb."""
    query_embedding = ipa2vec(query_ipa)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=["distances", "metadatas", "documents"] #include additional data
    )
    return results

##  with weighted levenshtein edit distance:-

### with alt ipa:-

In [26]:
def detect_language(text):
    """Detects if the input text is Hindi or English."""
    if any('\u0900' <= char <= '\u097F' for char in text):
        return 'hindi'
    else:
        return 'english'

In [1]:
from googletrans import Translator

# Initialize the Translator
translator = Translator()

def trans_hindi_to_english(hindi_text):
    result = translator.translate(hindi_text, src='hi', dest='en')
    return result.text

def trans_english_to_hindi(english_text):
    result = translator.translate(english_text, src='en', dest='hi')
    return result.text
print(trans_english_to_hindi("noel"))

नोएल


In [25]:
def querrier(name, n_results=50, weight_chromadb=0.7, weight_edit_distance=0.3, cutoff=0.6):
    """Retrieves similar names with combined ChromaDB and edit distance scoring."""
    language = detect_language(name)
    if language == 'hindi':
        query_ipa = hindi_to_ipa(name)
        alt_ipa_list = alt_ipa(query_ipa)
    else:
        query_ipa = english_to_ipa(name)
        alt_ipa_list = alt_ipa(name)
    print(f"alt_ipa_list: {alt_ipa_list}")
    all_scores = []

    # Process the main query IPA first:-
    main_query_embedding = ipa2vec(query_ipa)
    main_results = collection.query(
        query_embeddings=[main_query_embedding],
        n_results=n_results,
        include=["distances", "metadatas", "documents"]
    )
    for i in range(len(main_results["documents"][0])):
        chromadb_distance = main_results["distances"][0][i]
        metadata = main_results["metadatas"][0][i]
        doc_id = main_results["ids"][0][i]
        edit_distance = dst.weighted_feature_edit_distance(query_ipa, metadata["ipa"])
        combined_score = weight_chromadb * chromadb_distance + weight_edit_distance * edit_distance
        # if combined_score > cutoff:
        #     continue  # Skip results with a combined score greater than cutoff
        all_scores.append((main_results["documents"][0][i], metadata, chromadb_distance, combined_score, doc_id))
        print(f"Document: {main_results['documents'][0][i]}, ChromaDB Distance: {chromadb_distance}, Combined Score: {combined_score}, Weighted Edit Distance: {edit_distance}, ID: {doc_id}")
    # Process alternate IPAs:-
    weight_edit_distance=0
    weight_chromadb=1
    cutoff=0.003
    for alt_ipa_item in alt_ipa_list:
        alt_query_embedding = ipa2vec(alt_ipa_item)
        alt_results = collection.query(
            query_embeddings=[alt_query_embedding],
            n_results=n_results,
            include=["distances", "metadatas", "documents"]
        )
        for i in range(len(alt_results["documents"][0])):
            chromadb_distance = alt_results["distances"][0][i]
            metadata = alt_results["metadatas"][0][i]
            edit_distance = dst.weighted_feature_edit_distance(alt_ipa_item, metadata["ipa"])
            combined_score = weight_chromadb * chromadb_distance + weight_edit_distance * edit_distance
            # if combined_score > cutoff:
            #     continue  # Skip results with a combined score greater than cutoff
            all_scores.append((alt_results["documents"][0][i], metadata, chromadb_distance, combined_score))
            print(f"altDocument: {alt_results['documents'][0][i]}, ChromaDB Distance: {chromadb_distance}, Combined Score: {combined_score}, Weighted Edit Distance: {edit_distance}")
    # Sort all combined results by chromadb_distance (ascending order)
    all_scores.sort(key=lambda item: item[2])  # Sort by chromadb_distance

    # Extract top results
    filtered_results = {
        "documents": [item[0] for item in all_scores],
        "metadatas": [item[1] for item in all_scores],
        "distances": [item[2] for item in all_scores],
        "ids": [item[4] for item in all_scores],
    }
    # Remove duplicate elements in filtered_results:-
    unique_documents = []
    unique_metadatas = []
    unique_distances = []
    unique_ids = []

    for doc, meta, dist, doc_id in zip(filtered_results["documents"], filtered_results["metadatas"], filtered_results["distances"], filtered_results["ids"]):
        if doc not in unique_documents:
            unique_documents.append(doc)
            unique_metadatas.append(meta)
            unique_distances.append(dist)
            unique_ids.append(doc_id)

    filtered_results["documents"] = unique_documents
    filtered_results["metadatas"] = unique_metadatas
    filtered_results["distances"] = unique_distances
    filtered_results["ids"] = unique_ids
    return filtered_results

# wihtout alt ipa:

In [21]:
def querrier(name, n_results=100, weight_chromadb=0.7, weight_edit_distance=0.3, cutoff=0.6):
    """Retrieves similar names with combined ChromaDB and edit distance scoring."""
    language = detect_language(name)
    if language == 'hindi':
        query_ipa = hindi_to_ipa(name)
        print(query_ipa)
        # alt_ipa_list = alt_ipa(query_ipa)
    else:
        query_ipa = english_to_ipa(name)
        # alt_ipa_list = alt_ipa(name)
    # print(f"alt_ipa_list: {alt_ipa_list}")
    all_scores = []

    # Process the main query IPA first:-
    main_query_embedding = ipa2vec(query_ipa)
    main_results = collection.query(
        query_embeddings=[main_query_embedding],
        n_results=n_results,
        include=["distances", "metadatas", "documents"]
    )
    for i in range(len(main_results["documents"][0])):
        chromadb_distance = main_results["distances"][0][i]
        metadata = main_results["metadatas"][0][i]
        doc_id = main_results["ids"][0][i]
        edit_distance = dst.weighted_feature_edit_distance(query_ipa, metadata["ipa"])
        combined_score = weight_chromadb * chromadb_distance + weight_edit_distance * edit_distance
        if combined_score > cutoff:
            continue  # Skip results with a combined score greater than cutoff
        all_scores.append((main_results["documents"][0][i], metadata, chromadb_distance, combined_score, doc_id))
        print(f"Document: {main_results['documents'][0][i]}, ChromaDB Distance: {chromadb_distance}, Combined Score: {combined_score}, Weighted Edit Distance: {edit_distance}, ID: {doc_id}")
    # Process alternate IPAs:-
    # weight_edit_distance=0
    # weight_chromadb=1
    # cutoff=0.003
    # for alt_ipa_item in alt_ipa_list:
    #     alt_query_embedding = ipa2vec(alt_ipa_item)
    #     alt_results = collection.query(
    #         query_embeddings=[alt_query_embedding],
    #         n_results=n_results,
    #         include=["distances", "metadatas", "documents"]
    # #     )
    #     for i in range(len(alt_results["documents"][0])):
    #         chromadb_distance = alt_results["distances"][0][i]
    #         metadata = alt_results["metadatas"][0][i]
    #         edit_distance = dst.weighted_feature_edit_distance(alt_ipa_item, metadata["ipa"])
    #         combined_score = weight_chromadb * chromadb_distance + weight_edit_distance * edit_distance
    #         # if combined_score > cutoff:
    #         #     continue  # Skip results with a combined score greater than cutoff
    #         all_scores.append((alt_results["documents"][0][i], metadata, chromadb_distance, combined_score))
    #         print(f"altDocument: {alt_results['documents'][0][i]}, ChromaDB Distance: {chromadb_distance}, Combined Score: {combined_score}, Weighted Edit Distance: {edit_distance}")
    # Sort all combined results by chromadb_distance (ascending order)
    all_scores.sort(key=lambda item: item[2])  # Sort by chromadb_distance

    # Extract top results
    filtered_results = {
        "documents": [item[0] for item in all_scores],
        "metadatas": [item[1] for item in all_scores],
        "distances": [item[2] for item in all_scores],
        "ids": [item[4] for item in all_scores],
    }
    # Remove duplicate elements in filtered_results:-
    unique_documents = []
    unique_metadatas = []
    unique_distances = []
    unique_ids = []

    for doc, meta, dist, doc_id in zip(filtered_results["documents"], filtered_results["metadatas"], filtered_results["distances"], filtered_results["ids"]):
        if doc not in unique_documents:
            unique_documents.append(doc)
            unique_metadatas.append(meta)
            unique_distances.append(dist)
            unique_ids.append(doc_id)

    filtered_results["documents"] = unique_documents
    filtered_results["metadatas"] = unique_metadatas
    filtered_results["distances"] = unique_distances
    filtered_results["ids"] = unique_ids
    return filtered_results

# To input data to the database:-

## To input English data :-

## To enter large data:-(>40k)

### for multi part names:-

Resources:-

In [17]:
police_stations = [
    # Rewa District
    "Antrela", "Baikunthpur", "Chakghat", "Chorhata", "Civil Lines", "Dabhora", "Garh",
    "Govindgarh", "Hanumana", "Java", "Kotwali", "Laur", "Mangwan", "Mauganj",
    "Naigarhi", "Panwar", "Raipur", "Sagra", "Semariya", "Shahpur", "Sirmour",
    "Sohagi", "University", "Teonthar", "Traffic", "Mahila",
    
    # Sagar District
    "Kotwali", "Motinagar", "Cannt", "Makronia", "Baheria", "Civil Lines", "Gopal Ganj",
    "Women Police Station", "Banda", "Shahgarh", "Chhanbila", "Behrol", "Baryatha",
    "Binaika", "Dalpatpur", "SDOP Deori", "Deori", "Gaurjhawar", "Maharajpur", "Kesli",
    "SDOP Khurai", "Khurai", "Bandri", "Malthone", "SDOP Rehli", "Rehli", "Garhakota",
    "Sanodha", "Surkhi", "SDOP Rahatgarh", "Rahatgarh", "Jaisinagar", "Naryaoli",
    "SDOP Bina", "Bina", "Bhangarh", "Khimlasa", "Agasod",

    # Indore District
    "Gandhi Nagar", "Goutampura", "Hatod", "Hira Nagar", "Juni Indore", "Kanadiya",
    "Khajrana", "Khudail", "Kishanganj", "Kotwali", "Kshipra", "Lasudia", 
    "Mahila Thana", "Malharganj", "Manpur", "Mhow", "MIG", "Palasiya", 
    "Pandrinath", "Pardeshipura", "Rajendra Nagar", "Raoji Bazar", "Rau", 
    "Sadar Bazar", "Sanwer", "Sarafa", "Sayogita Ganj", "Simrol", 
    "Tejaji Nagar", "Tilak Nagar", "Tukoganj", "Vijay Nagar"
]

ipc_crimes = [
    "Murder - IPC Section 302",
    "Attempt to Murder - IPC Section 307",
    "Culpable Homicide - IPC Section 304",
    "Rape - IPC Section 376",
    "Dowry Death - IPC Section 304B",
    "Kidnapping - IPC Section 361",
    "Theft - IPC Section 378",
    "Extortion - IPC Section 383",
    "Robbery - IPC Section 392",
    "Dacoity - IPC Section 395",
    "Criminal Breach of Trust - IPC Section 406",
    "Cheating - IPC Section 415",
    "Forgery - IPC Section 463",
    "Defamation - IPC Section 499",
    "Causing Grievous Hurt - IPC Section 320",
    "Criminal Intimidation - IPC Section 506",
    "Rioting - IPC Section 147",
    "Unlawful Assembly - IPC Section 141",
    "Counterfeiting Currency - IPC Section 489A",
    "Adultery (Decriminalized) - IPC Section 497",
    "Acid Attack - IPC Section 326A",
    "Abetment of Suicide - IPC Section 306",
    "Abduction - IPC Section 362",
    "Arson - IPC Section 436",
    "Bigamy - IPC Section 494",
    "Bribery - IPC Section 171B",
    "Cruelty to Women - IPC Section 498A",
    "Embezzlement - IPC Section 403",
    "False Evidence - IPC Section 191",
    "Fraud - IPC Section 420",
    "Harassment - IPC Section 509",
    "Hurt - IPC Section 319",
    "Indecent Representation of Women - IPC Section 66E",
    "Kidnapping for Ransom - IPC Section 364A",
    "Mischief - IPC Section 425",
    "Pawning of Stolen Goods - IPC Section 410",
    "Perjury - IPC Section 191",
    "Sexual Harassment - IPC Section 354",
    "Sedition - IPC Section 124A",
    "Terrorism - IPC Section 121",
    "Trespass - IPC Section 441",
    "Unlawful Assembly - IPC Section 141",
    "Waging War Against the Government - IPC Section 121",
]
addresses = [
    "25, Shivaji Marg, Indore, Madhya Pradesh - 452001",
    "Plot No. 14, Sector C, Govindpura, Bhopal, Madhya Pradesh - 462023",
    "78, Station Road, Jabalpur, Madhya Pradesh - 482001",
    "House No. 128, Vijay Nagar, Gwalior, Madhya Pradesh - 474011",
    "Shop No. 5, MG Road, Ujjain, Madhya Pradesh - 456001",
    "102, New Market, TT Nagar, Bhopal, Madhya Pradesh - 462003",
    "56, Patel Nagar, Ratlam, Madhya Pradesh - 457001",
    "Flat No. 401, Emerald Heights, Indore, Madhya Pradesh - 452010",
    "Plot No. 89, Shanti Nagar, Rewa, Madhya Pradesh - 486001",
    "Shop No. 12, Main Bazaar, Satna, Madhya Pradesh - 485001",
    "House No. 45, Sindhi Colony, Sagar, Madhya Pradesh - 470001",
    "91, Nehru Ward, Damoh, Madhya Pradesh - 470661",
    "Shop No. 10, Bus Stand Road, Chhindwara, Madhya Pradesh - 480001",
    "H.No. 120, Balaghat Road, Seoni, Madhya Pradesh - 480661",
    "123, Gandhi Nagar, Khargone, Madhya Pradesh - 451001",
    "Plot No. 35, Narmada Colony, Khandwa, Madhya Pradesh - 450001",
    "78, Jawahar Chowk, Harda, Madhya Pradesh - 461331",
    "Flat No. 202, Sai Residency, Hoshangabad, Madhya Pradesh - 461001",
    "Shop No. 7, Bada Bazaar, Vidisha, Madhya Pradesh - 464001",
    "Plot No. 56, Saraswati Nagar, Raisen, Madhya Pradesh - 464551",
    "House No. 22, Shastri Colony, Betul, Madhya Pradesh - 460001",
    "Plot No. 90, Navjeevan Colony, Dewas, Madhya Pradesh - 455001",
    "Shop No. 18, Gandhi Chowk, Mandsaur, Madhya Pradesh - 458001",
    "H.No. 304, Shanti Vihar, Neemuch, Madhya Pradesh - 458441",
    "Plot No. 65, Shankar Nagar, Morena, Madhya Pradesh - 476001",
    "Flat No. 102, Galaxy Heights, Bhind, Madhya Pradesh - 477001",
    "House No. 78, Ram Nagar, Tikamgarh, Madhya Pradesh - 472001",
    "Plot No. 45, Laxmi Nagar, Chhatarpur, Madhya Pradesh - 471001",
    "Shop No. 3, Main Road, Panna, Madhya Pradesh - 488001",
    "House No. 110, Saraswati Colony, Shahdol, Madhya Pradesh - 484001",
    "Plot No. 76, Green Valley, Anuppur, Madhya Pradesh - 484444",
    "H.No. 55, Jai Stambh Chowk, Singrauli, Madhya Pradesh - 486889",
    "Shop No. 21, Market Road, Umaria, Madhya Pradesh - 484661",
    "Plot No. 13, Industrial Area, Katni, Madhya Pradesh - 483501",
    "H.No. 99, Surya Nagar, Balaghat, Madhya Pradesh - 481001",
    "Shop No. 8, Collectorate Road, Mandla, Madhya Pradesh - 481661",
    "Plot No. 120, Malviya Nagar, Sehore, Madhya Pradesh - 466001",
    "Flat No. 405, Harmony Apartments, Ashoknagar, Madhya Pradesh - 473331",
    "Shop No. 11, Rajwada, Datia, Madhya Pradesh - 475661",
    "House No. 67, Kamla Nagar, Shivpuri, Madhya Pradesh - 473551"
]
common_indian_surnames = [
    "Agarwal", "Ahuja", "Anand", "Apte", "Arya", "Asthana", "Bachchan", "Bagchi", "Bajaj", "Balakrishnan",
    "Banerjee", "Bansal", "Barua", "Basu", "Batla", "Bedi", "Behera", "Bhalla", "Bhandari", "Bhat",
    "Bhatia", "Bhatt", "Bhattacharya", "Biswas", "Bose", "Chakrabarti", "Chanda", "Chandra", "Chatterjee",
    "Chaudhary", "Chauhan", "Chhabra", "Chopra", "Dalal", "Das", "Dash", "Datta", "Desai", "Deshmukh",
    "Devadiga", "Dey", "Dhaliwal", "Dhar", "Dhillon", "Dixit", "Dubey", "Dutta", "Gandhi", "Garg",
    "Gaur", "Ghosh", "Gill", "Goel", "Gopal", "Goswami", "Goud", "Gowda", "Guha", "Gupta", "Halder",
    "Iyengar", "Iyer", "Jain", "Jaiswal", "Jat", "Jha", "Joshi", "Kadam", "Kapoor", "Kar", "Kashyap",
    "Kaur", "Khan", "Khanna", "Khare", "Khatri", "Kochhar", "Kohli", "Konar", "Kulkarni", "Kumar",
    "Lahiri", "Lal", "Mahajan", "Maharaj", "Maheshwari", "Malhotra", "Malik", "Mandal", "Marathe",
    "Mehra", "Mehta", "Meena", "Menon", "Mishra", "Mitra", "Mittal", "Modi", "Mudaliar", "Mukherjee",
    "Nadar", "Nag", "Nagar", "Naidu", "Nair", "Nanda", "Narang", "Narayan", "Nath", "Nayak", "Nayyar",
    "Negi", "Nehru", "Nigam", "Pal", "Pande", "Pandey", "Pant", "Parida", "Parikh", "Patel", "Pathak",
    "Patil", "Pawar", "Pillai", "Pooniwala", "Prabhu", "Prasad", "Puri", "Rai", "Raj", "Rajan", "Rajput",
    "Ram", "Raman", "Rao", "Rastogi", "Rathore", "Raut", "Rawat", "Reddy", "Sable", "Sachdev", "Saha",
    "Sahu", "Saini", "Salvi", "Samaddar", "Sandhu", "Sankar", "Sant", "Sapra", "Sarangi", "Sarkar",
    "Sarma", "Saxena", "Sehgal", "Sen", "Sengupta", "Seth", "Setty", "Shah", "Shaikh", "Sharma",
    "Shenoy", "Shetty", "Shroff", "Shukla", "Siddhu", "Singh", "Sinha", "Somaiya", "Sonawane", "Sood",
    "Srivastava", "Subramaniam", "Subramanian", "Sundaram", "Swamy", "Talwar", "Tandon", "Thaker",
    "Thakur", "Thatte", "Tiwari", "Tripathi", "Trivedi", "Varshney", "Venkatesh", "Verma", "Yadav", "Zachariah",
]

In [10]:
from googletrans import Translator

# Initialize the Translator
translator = Translator()

def trans_hindi_to_english(hindi_text):
    result = translator.translate(hindi_text, src='hi', dest='en')
    return result.text

def trans_english_to_hindi(english_text):
    result = translator.translate(english_text, src='en', dest='hi')
    return result.text
print(trans_english_to_hindi("noel"))

नोएल


In [18]:
def detect_language(text):
    """Detects if the input text is Hindi or English."""
    if any('\u0900' <= char <= '\u097F' for char in text):
        return 'hindi'
    else:
        return 'english'

In [16]:
import random
import math
import re
import numpy as np
from datetime import datetime, timedelta

# Define the file path
input_path = "fuzzified56k(200k).txt"

# Read names from file
with open(input_path, "r") as file:
    names = [line.strip() for line in file.readlines()]

# Function to vectorize name parts
def vectorize_name_parts(name):
    # Remove special characters
    cleaned_name = re.sub(r'[^\w\s]|[\d]', '', name)
    parts = [part for part in cleaned_name.split() if len(part) > 2]
    ipa_parts = [english_to_ipa(part) for part in parts]
    vectors = [ipa2vec(ipa) for ipa in ipa_parts]
    return parts, ipa_parts, vectors

# Function to generate a random Aadhaar card number
def generate_aadhaar():
    return ''.join([str(random.randint(0, 9)) for _ in range(12)])

# Function to generate a random date of birth and date of crime
def generate_dob_and_doc():
    start_date_dob = datetime.strptime('1950-01-01', '%Y-%m-%d')
    end_date_dob = datetime.strptime('2005-12-31', '%Y-%m-%d')
    random_date_dob = start_date_dob + timedelta(days=random.randint(0, (end_date_dob - start_date_dob).days))
    dob = random_date_dob.strftime('%d-%m-%Y')
    age = datetime.now().year - random_date_dob.year

    end_date_doc = datetime.now()
    start_date_doc = min(random_date_dob + timedelta(days=365 * 20), end_date_doc)  # Ensure doc is at least 20 years after dob
    if start_date_doc < end_date_doc:
        random_date_doc = start_date_doc + timedelta(days=random.randint(0, (end_date_doc - start_date_doc).days))
    else:
        random_date_doc = start_date_doc
    doc = random_date_doc.strftime('%d-%m-%Y')

    return dob, age, doc

# Calculate the number of batches
batch_size = 1000  # Choose a batch size less than the maximum
num_batches = math.ceil(len(names) / batch_size)

# Add data in batches
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(names))

    batch_names = names[start_idx:end_idx]
    batch_embeddings = []
    batch_metadatas = []
    batch_ids = []
    for name_idx, name in enumerate(batch_names):
        # Append a random surname
        surname = random.choice(common_indian_surnames)
        full_name = f"{name} {surname}"
        parts, ipa_parts, vectors = vectorize_name_parts(full_name)
        if vectors:
            type = random.choice(["Criminal", "Suspect", "Victim", "Witness"])
            aadhaar = generate_aadhaar()
            dob, age, doc = generate_dob_and_doc()
            gender = 'Male'  # Randomly assign gender
            police_station = random.choice(police_stations)  # Randomly select a police station
            crime = random.choice(ipc_crimes)  # Randomly select a crime
            address = random.choice(addresses)  # Randomly select an address
            for part_idx, (part, ipa, vector) in enumerate(zip(parts, ipa_parts, vectors)):
                if vector:
                    batch_embeddings.append(vector)
                    batch_metadatas.append({"full_name": full_name, "part": part, "ipa": ipa, "age": age, "aadhaar": aadhaar, "gender": gender, "dob": dob, "doc": doc, "station": police_station, "crime": crime, "address": address, "type": type,"trans_name":trans_name})
                    batch_ids.append(f"{start_idx + name_idx}_{part_idx}")
                else:
                    reason = "is nan" 
                    print(f"Skipping vector for part '{part}' in name '{full_name}'({reason}) due to invalid vector: {vector} ")
    # print(f"Batch {i + 1} embeddings sample: {batch_embeddings[:4]}")
    if batch_embeddings:  # Ensure there is data to add
        collection.add(
            documents=[metadata["full_name"] for metadata in batch_metadatas],
            embeddings=batch_embeddings,
            ids=batch_ids,
            metadatas=batch_metadatas
        )
        print(f"Added batch {i + 1} of {num_batches}")


Added batch 1 of 202
Added batch 2 of 202
Added batch 3 of 202
Added batch 4 of 202
Added batch 5 of 202
Added batch 6 of 202
Added batch 7 of 202
Added batch 8 of 202
Added batch 9 of 202
Added batch 10 of 202
Added batch 11 of 202
Added batch 12 of 202
Added batch 13 of 202
Added batch 14 of 202
Added batch 15 of 202
Added batch 16 of 202
Added batch 17 of 202
Added batch 18 of 202
Added batch 19 of 202
Added batch 20 of 202
Added batch 21 of 202
Added batch 22 of 202
Added batch 23 of 202
Added batch 24 of 202
Added batch 25 of 202
Added batch 26 of 202
Added batch 27 of 202
Added batch 28 of 202
Added batch 29 of 202
Added batch 30 of 202
Added batch 31 of 202
Added batch 32 of 202
Added batch 33 of 202
Added batch 34 of 202
Added batch 35 of 202
Added batch 36 of 202
Added batch 37 of 202
Added batch 38 of 202
Added batch 39 of 202
Added batch 40 of 202
Added batch 41 of 202
Added batch 42 of 202
Added batch 43 of 202
Added batch 44 of 202
Added batch 45 of 202
Added batch 46 of 2

chroma won't allow u to add >40k entries in a single add request, so u have to do it in batches and also think about fixing the unique id problem

## For hindi singles:-

In [19]:
import random
import math
import re
import numpy as np
from datetime import datetime, timedelta

# Define the file path
input_path = "fuzzified56k(200k).txt"

# Read names from file
with open(input_path, "r") as file:
    names = [line.strip() for line in file.readlines()]

# Read Hindi names from file
hindi_input_path = "hindiv3.txt"
with open(hindi_input_path, "r") as file:
    hindi_names = [line.strip() for line in file.readlines()]

# Function to vectorize a single-word name
def vectorize_single_word_name(name):
    # Remove special characters
    cleaned_name = re.sub(r'[^\w\s]|[\d]', '', name)
    ipa = english_to_ipa(cleaned_name)
    vector = ipa2vec(ipa)
    return cleaned_name, ipa, vector

# Function to generate a random Aadhaar card number
def generate_aadhaar():
    return ''.join([str(random.randint(0, 9)) for _ in range(12)])

# Function to generate a random date of birth and date of crime
def generate_dob_and_doc():
    start_date_dob = datetime.strptime('1950-01-01', '%Y-%m-%d')
    end_date_dob = datetime.strptime('2005-12-31', '%Y-%m-%d')
    random_date_dob = start_date_dob + timedelta(days=random.randint(0, (end_date_dob - start_date_dob).days))
    dob = random_date_dob.strftime('%d-%m-%Y')
    age = datetime.now().year - random_date_dob.year

    end_date_doc = datetime.now()
    start_date_doc = min(random_date_dob + timedelta(days=365 * 20), end_date_doc)  # Ensure doc is at least 20 years after dob
    if start_date_doc < end_date_doc:
        random_date_doc = start_date_doc + timedelta(days=random.randint(0, (end_date_doc - start_date_doc).days))
    else:
        random_date_doc = start_date_doc
    doc = random_date_doc.strftime('%d-%m-%Y')

    return dob, age, doc

# Calculate the number of batches
batch_size = 10000  # Choose a batch size less than the maximum
num_batches = math.ceil(len(names) / batch_size)

# Add data in batches
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(names))

    batch_names = names[start_idx:end_idx]
    batch_hindi_names = hindi_names[start_idx:end_idx]
    batch_embeddings = []
    batch_metadatas = []
    batch_ids = []
    for name_idx, (name, hindi_name) in enumerate(zip(batch_names, batch_hindi_names)):
        cleaned_name, ipa, vector = vectorize_single_word_name(name)
        if vector:
            type = random.choice(["Criminal", "Suspect", "Victim", "Witness"])
            aadhaar = generate_aadhaar()
            dob, age, doc = generate_dob_and_doc()
            gender = 'Male'  # Randomly assign gender
            police_station = random.choice(police_stations)  # Randomly select a police station
            crime = random.choice(ipc_crimes)  # Randomly select a crime
            address = random.choice(addresses)  # Randomly select an address
            batch_embeddings.append(vector)
            batch_metadatas.append({"full_name": cleaned_name, "ipa": ipa, "age": age, "aadhaar": aadhaar, "gender": gender, "dob": dob, "doc": doc, "station": police_station, "crime": crime, "address": address, "type": type, "trans_name": hindi_name})
            batch_ids.append(f"{start_idx + name_idx}")
        else:
            reason = "is nan" 
            print(f"Skipping vector for name '{cleaned_name}'({reason}) due to invalid vector: {vector}")
    # print(f"Batch {i + 1} embeddings sample: {batch_embeddings[:4]}")
    if batch_embeddings:  # Ensure there is data to add
        collection.add(
            documents=[metadata["full_name"] for metadata in batch_metadatas],
            embeddings=batch_embeddings,
            ids=batch_ids,
            metadatas=batch_metadatas
        )
        print(f"Added batch {i + 1} of {num_batches}")


Skipping vector for name ''(is nan) due to invalid vector: []
Added batch 1 of 21
Added batch 2 of 21
Added batch 3 of 21
Added batch 4 of 21
Added batch 5 of 21
Added batch 6 of 21
Added batch 7 of 21
Added batch 8 of 21
Added batch 9 of 21
Added batch 10 of 21
Added batch 11 of 21
Skipping vector for name ''(is nan) due to invalid vector: []
Skipping vector for name ''(is nan) due to invalid vector: []
Skipping vector for name ''(is nan) due to invalid vector: []
Skipping vector for name ''(is nan) due to invalid vector: []
Skipping vector for name ''(is nan) due to invalid vector: []
Added batch 12 of 21
Skipping vector for name ''(is nan) due to invalid vector: []
Skipping vector for name ''(is nan) due to invalid vector: []
Skipping vector for name ''(is nan) due to invalid vector: []
Skipping vector for name ''(is nan) due to invalid vector: []
Added batch 13 of 21
Added batch 14 of 21
Added batch 15 of 21
Added batch 16 of 21
Added batch 17 of 21
Added batch 18 of 21
Added batch

### single name only:- 

In [ ]:
import random
import math
import re
import numpy as npz
from datetime import datetime, timedelta

# Define the file path
input_path = "fuzzified56k(200k).txt"

# Read names from file
with open(input_path, "r") as file:
    names = [line.strip() for line in file.readlines()]

# Function to vectorize name parts
def vectorize_name_parts(name):
    # Remove special characters
    cleaned_name = re.sub(r'[^\w\s]|[\d]', '', name)
    parts = [part for part in cleaned_name.split() if len(part) > 2]
    ipa_parts = [english_to_ipa(part) for part in parts]
    vectors = [ipa2vec(ipa) for ipa in ipa_parts]
    return parts, ipa_parts, vectors

# Function to generate a random Aadhaar card number
def generate_aadhaar():
    return ''.join([str(random.randint(0, 9)) for _ in range(12)])

# Function to generate a random date of birth and date of crime
def generate_dob_and_doc():
    start_date_dob = datetime.strptime('1950-01-01', '%Y-%m-%d')
    end_date_dob = datetime.strptime('2005-12-31', '%Y-%m-%d')
    random_date_dob = start_date_dob + timedelta(days=random.randint(0, (end_date_dob - start_date_dob).days))
    dob = random_date_dob.strftime('%d-%m-%Y')
    age = datetime.now().year - random_date_dob.year

    end_date_doc = datetime.now()
    start_date_doc = min(random_date_dob + timedelta(days=365 * 20), end_date_doc)  # Ensure doc is at least 20 years after dob
    if start_date_doc < end_date_doc:
        random_date_doc = start_date_doc + timedelta(days=random.randint(0, (end_date_doc - start_date_doc).days))
    else:
        random_date_doc = start_date_doc
    doc = random_date_doc.strftime('%d-%m-%Y')

    return dob, age, doc


# Set to keep track of unique parts
unique_parts = set()
# Calculate the number of batches
batch_size = 10000  # Choose a batch size less than the maximum
num_batches = math.ceil(len(names) / batch_size)

# Add data in batches
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(names))

    batch_names = names[start_idx:end_idx]
    batch_embeddings = []
    batch_metadatas = []
    batch_ids = []
    for name_idx, name in enumerate(batch_names):
        parts, ipa_parts, vectors = vectorize_name_parts(name)
        for part_idx, (part, ipa, vector) in enumerate(zip(parts, ipa_parts, vectors)):
            part = part.title()
            if part not in unique_parts:
                unique_parts.add(part)
                aadhaar = generate_aadhaar()
                dob, age, doc = generate_dob_and_doc()
                gender = 'Male'  # Randomly assign gender
                police_station = random.choice(police_stations)  # Randomly select a police station
                crime = random.choice(ipc_crimes)  # Randomly select a crime
                address = random.choice(addresses)  # Randomly select an address
                batch_embeddings.append(vector)
                batch_metadatas.append({"full_name": name, "part": part, "ipa": ipa, "age": age, "aadhaar": aadhaar, "gender": gender, "dob": dob, "doc": doc, "station": police_station, "crime": crime, "address": address})
                batch_ids.append(f"{start_idx + name_idx}_{part_idx}")

    # print(f"Batch {i + 1} embeddings sample: {batch_embeddings[:4]}")
    if batch_embeddings:  # Ensure there is data to add
        collection.add(
            documents=[metadata["part"] for metadata in batch_metadatas],
            embeddings=batch_embeddings,
            ids=batch_ids,
            metadatas=batch_metadatas
        )
        print(f"Added batch {i + 1} of {num_batches}")


Added batch 1 of 21
Added batch 2 of 21
Added batch 3 of 21
Added batch 4 of 21
Added batch 5 of 21
Added batch 6 of 21
Added batch 7 of 21
Added batch 8 of 21
Added batch 9 of 21
Added batch 10 of 21
Added batch 11 of 21
Added batch 12 of 21
Added batch 13 of 21
Added batch 14 of 21
Added batch 15 of 21
Added batch 16 of 21
Added batch 17 of 21
Added batch 18 of 21
Added batch 19 of 21
Added batch 20 of 21
Added batch 21 of 21


## Else:-

In [ ]:
# Add pronunciations to the ChromaDB collection
embedder(names, ipas)

# all_embeddings = collection.get(include=["embeddings"])["embeddings"]
# print("All embeddings:", all_embeddings)

### To input hindi data :-

In [ ]:
# Define the file path
input_path = "sname.txt"
# Read names from file
with open(input_path, "r") as file:
    names = [line.strip() for line in file.readlines()]

# Convert names to IPA
ipas = [hindi_to_ipa(name) for name in names]

# Add pronunciations to the ChromaDB collection
embedder(names, ipas)

all_embeddings = collection.get(include=["embeddings"])["embeddings"]
# print("All embeddings:", all_embeddings)

# Querry:-

In [22]:
query_name = "laxmi"
results = querrier(query_name)
print(results['documents'])
print(results['metadatas'])
print(results['distances'])

Document: Laxmi, ChromaDB Distance: 0.0, Combined Score: 0.0, Weighted Edit Distance: 0.0, ID: 19602
Document: Laxmee, ChromaDB Distance: 0.0, Combined Score: 0.0, Weighted Edit Distance: 0.0, ID: 48158
Document: Laxmii, ChromaDB Distance: 0.0, Combined Score: 0.0, Weighted Edit Distance: 0.0, ID: 48233
Document: Laxhmi, ChromaDB Distance: 0.0, Combined Score: 0.0, Weighted Edit Distance: 0.0, ID: 48230
Document: Lakhsmi, ChromaDB Distance: 0.0, Combined Score: 0.0, Weighted Edit Distance: 0.0, ID: 48281
Document: Laxmmi, ChromaDB Distance: 0.0, Combined Score: 0.0, Weighted Edit Distance: 0.0, ID: 48244
Document: Laxmy, ChromaDB Distance: 0.0, Combined Score: 0.0, Weighted Edit Distance: 0.0, ID: 48239
Document: Laksmi, ChromaDB Distance: 0.0, Combined Score: 0.0, Weighted Edit Distance: 0.0, ID: 154969
Document: Lakhshmi, ChromaDB Distance: 0.0, Combined Score: 0.0, Weighted Edit Distance: 0.0, ID: 155820
Document: laxmi, ChromaDB Distance: 0.0, Combined Score: 0.0, Weighted Edit Dis

In [19]:
# List of 20 Indian names
indian_names = [
    "Aarav", "Vivaan", "Aditya", "Vihaan", "Arjun",
    "Sai", "Reyansh", "Ayaan", "Krishna", "Ishaan",
    "Shaurya", "Atharv", "Dhruv", "Kabir", "Rudra",
    "Aarush", "Anay", "Om", "Parth", "Rishi"
]

# Loop through each name, convert to IPA, and query the ChromaDB collection
for name in indian_names:
    ipa = english_to_ipa(name)
    print(f"The IPA representation of '{name}' is: {ipa}")
    results = querrier(ipa)
    print(f"Results for '{name}':")
    print(results['documents'])
    # print(results['metadatas'])
    print("distance:-",results['distances'])
indian_names = [
    "Aarav", "Vivaan", "Aditya", "Vihaan", "Arjun",
    "Sai", "Reyansh", "Ayaan", "Krishna", "Ishaan",
    "Shaurya", "Atharv", "Dhruv", "Kabir", "Rudra",
    "Aarush", "Anay", "Om", "Parth", "Rishi"
]


The IPA representation of 'Aarav' is: æɹɑv
Document: Aaravi, ChromaDB Distance: -1.1920928955078125e-07, Combined Score: -8.344650268554687e-08, Weighted Edit Distance: 0.0, ID: 116421_0
Document: Aarabi, ChromaDB Distance: 0.013513386249542236, Combined Score: 0.3844593703746796, Weighted Edit Distance: 1.25, ID: 116334_0
Document: Aaraabi, ChromaDB Distance: 0.013513386249542236, Combined Score: 0.3844593703746796, Weighted Edit Distance: 1.25, ID: 116336_0
Document: Arivi, ChromaDB Distance: 0.0194321870803833, Combined Score: 0.5386025309562683, Weighted Edit Distance: 1.75, ID: 119829_0
Results for 'Aarav':
['Aaravi', 'Aarabi', 'Aaraabi', 'Arivi']
distance:- [-1.1920928955078125e-07, 0.013513386249542236, 0.013513386249542236, 0.0194321870803833]
The IPA representation of 'Vivaan' is: vɪvɑn
Results for 'Vivaan':
[]
distance:- []
The IPA representation of 'Aditya' is: ɑdɪtjə
Results for 'Aditya':
[]
distance:- []
The IPA representation of 'Vihaan' is: vihɑn
Results for 'Vihaan':
[]

In [50]:
# Get user input
# example :- Enter a name in Hindi:अधि
name = input("Enter a name in Hindi: ")

# Convert the name to IPA
ipa = hindi_to_ipa(name)

print(f"The IPA representation of '{name}' is: {ipa}")

# Query the ChromaDB collection
results = chromaquerrier(ipa)
print(results['documents'])
print(results['metadatas'])
print(results['distances'])

The IPA representation of 'गौतमी' is: ɡɔːtmiː
[['Agampreet', 'Muttuk Kumaran', 'Kumbhkarna', 'Kruban', 'Pragnika']]
[[{'age': 41, 'ipa': 'əɡəmpɹit'}, {'age': 70, 'ipa': 'mʌtək kumɹ̩æn'}, {'age': 22, 'ipa': 'kʌmbkɑɹnə'}, {'age': 37, 'ipa': 'kɹʌbən'}, {'age': 42, 'ipa': 'pɹæɡnɪkə'}]]
[[0.8537500500679016, 0.9203306436538696, 0.9204938411712646, 0.9377778768539429, 0.9412500262260437]]


In [ ]:
# Get user input
# example :- Enter a name in Hindi:अधि
name = input("Enter a name in English: ")

# Convert the name to IPA
ipa = hindi_to_ipa(name)

print(f"The IPA representation of '{name}' is: {ipa}")

# Query the ChromaDB collection
results = chromaquerrier(ipa)
print(results['documents'])
print(results['metadatas'])
print(results['distances'])